In [1]:
import os
import numpy
from datetime import datetime, timedelta
import re
import xarray
import bz2

In [2]:
current_folder = '/data01/people/beichen/workspace/20240117'
ahi_ac_timelist_npy = os.path.join(current_folder, 'MYD_RAA_AU_30S_5-8_clear_ahi_time.npy')
ahi_ac_timelist = numpy.load(ahi_ac_timelist_npy)

year_first_day_date = datetime.strptime('2018-01-01T00:00:00Z', "%Y-%m-%dT%H:%M:%SZ")

ROI_SIZE = 0.04
# VZA diff
DIFF_VZA_THRESHOLD = 1 # degree
# RAA diff
DIFF_RAA_THRESHOLD = 10 # degree

AHI_VZA_BIN = '/data01/people/beichen/data/AHI/AHI_VZA.npy'
AHI_VAA_BIN = '/data01/people/beichen/data/AHI/AHI_VAA.npy'

AHI_ANGLE_FOLDER = '/data01/people/beichen/data/AHI_ANGLE2017010120201231/hmwr829gr.cr.chiba-u.ac.jp/gridded/FD/V20190123'

DATA_FOLDER = '/data01/people/beichen/data'
MXD_GEOTIFF_FOLDER = os.path.join(DATA_FOLDER, 'MYD03_AHI_GeoTiff_2018')

In [3]:
def get_region_ahi_vza(n_lons, n_lats):
    ahi_vza_DN = numpy.load(AHI_VZA_BIN, allow_pickle=True)
    p_size = 120 / 3000
    lons_4km = numpy.arange(85+p_size/2, 205, p_size)
    lats_4km = numpy.arange(60-p_size/2, -60, -p_size)
    
    o_ds = xarray.Dataset(
        data_vars={
            "values": (("y", "x"), ahi_vza_DN),
        },
        coords={
            "y": lats_4km,
            "x": lons_4km
        },)
    n_ex_ds = o_ds.interp(x=n_lons, y=n_lats, method="nearest", kwargs={"fill_value": "extrapolate"})
    return n_ex_ds["values"]


def get_region_ahi_vaa(n_lons, n_lats):
    ahi_vaa_DN = numpy.load(AHI_VAA_BIN, allow_pickle=True)
    p_size = 120 / 3000
    lons_4km = numpy.arange(85+p_size/2, 205, p_size)
    lats_4km = numpy.arange(60-p_size/2, -60, -p_size)
    
    o_ds = xarray.Dataset(
        data_vars={
            "values": (("y", "x"), ahi_vaa_DN),
        },
        coords={
            "y": lats_4km,
            "x": lons_4km
        },)
    n_ex_ds = o_ds.interp(x=n_lons, y=n_lats, method="nearest", kwargs={"fill_value": "extrapolate"})
    return n_ex_ds["values"]


def get_region_ahi_saa(ahi_saa_file, n_lons, n_lats):
    zipfile = bz2.BZ2File(ahi_saa_file)
    ahi_saa = numpy.frombuffer(zipfile.read(), dtype='>f4').reshape((3000,3000))
    p_size = 120 / 3000
    lons_4km = numpy.arange(85+p_size/2, 205, p_size)
    lats_4km = numpy.arange(60-p_size/2, -60, -p_size)
    
    o_ds = xarray.Dataset(
        data_vars={
            "values": (("y", "x"), ahi_saa),
        },
        coords={
            "y": lats_4km,
            "x": lons_4km
        },)
    n_ex_ds = o_ds.interp(x=n_lons, y=n_lats, method="nearest", kwargs={"fill_value": "extrapolate"})
    return n_ex_ds["values"]


def get_region_ahi_sza(ahi_sza_file, n_lons, n_lats):
    zipfile = bz2.BZ2File(ahi_sza_file)
    ahi_sza = numpy.frombuffer(zipfile.read(), dtype='>f4').reshape((3000,3000))
    p_size = 120 / 3000
    lons_4km = numpy.arange(85+p_size/2, 205, p_size)
    lats_4km = numpy.arange(60-p_size/2, -60, -p_size)
    
    o_ds = xarray.Dataset(
        data_vars={
            "values": (("y", "x"), ahi_sza),
        },
        coords={
            "y": lats_4km,
            "x": lons_4km
        },)
    n_ex_ds = o_ds.interp(x=n_lons, y=n_lats, method="nearest", kwargs={"fill_value": "extrapolate"})
    return n_ex_ds


def regex_match_substring(pattern, array):
    matches = [element for element in array if re.search(pattern, element)]

    return matches


def clip_tif_studyarea_angle(tiff_filename, lons, lats):
    tif_ds = xarray.open_rasterio(tiff_filename)
    clip_ds = tif_ds.interp(x=lons, y=lats, method="nearest")
    angle_dn = numpy.array(clip_ds.values[0])
    angle_dn[angle_dn==-32767] = numpy.NaN
    angle_v = angle_dn/100
    angle_v = (angle_v+360)%360
    clip_ds.values[0] = angle_v
    return clip_ds


def nearest_10_minute_interval(given_time):
    given_datetime = datetime.strptime(given_time, '%Y%m%d%H%M')

    nearest_interval = (given_datetime + timedelta(minutes=5)).replace(second=0, microsecond=0)
    nearest_interval = nearest_interval - timedelta(minutes=nearest_interval.minute % 10)

    return nearest_interval


def get_raa(aa1, aa2):
    diff = numpy.abs(aa1 - aa2)
    raa = numpy.where(diff < 180, diff, 360 - diff)
    return raa

def is_raa_matched(modis_vza, modis_vaa, modis_saa, ahi_vza, ahi_vaa, ahi_saa):
    vza_condition = numpy.abs(modis_vza - ahi_vza) <= DIFF_VZA_THRESHOLD
    modis_raa = get_raa(modis_vaa, modis_saa)
    ahi_raa = get_raa(ahi_vaa, ahi_saa)
    raa_condition = numpy.abs(modis_raa - ahi_raa) <= DIFF_RAA_THRESHOLD
    return numpy.logical_and(vza_condition, raa_condition)

In [4]:
ahi_time_list4match = numpy.array(ahi_ac_timelist)[:,2]
ahi_time_list4match_u = numpy.unique(ahi_time_list4match)

In [5]:
ahi_day4match = []
for ahi_time_str in ahi_time_list4match_u:
    ahi_time_str_day = ahi_time_str[:8]
    ahi_day4match.append(ahi_time_str_day)
ahi_day4match_u = numpy.unique(numpy.array(ahi_day4match))

In [6]:
sample_internal = 0.1
AU_extent = [110, -28, 125, -38] # llon, llat, rlon, rlat
study_area_extent = [ -28, 110, -38, 125] # extent (ullat, ullon, lrlat, lrlon)

AU_lons_10km = numpy.arange(110+sample_internal/2, 125, sample_internal)
AU_lats_10km = numpy.arange(-28-sample_internal/2, -38, -sample_internal)

clip_internal = 0.01
AU_lons_1km = numpy.arange(110+clip_internal/2, 125, clip_internal)
AU_lats_1km = numpy.arange(-28-clip_internal/2, -38, -clip_internal)

In [7]:
vza_ahi_au_ds = get_region_ahi_vza(AU_lons_1km, AU_lats_1km)
vaa_ahi_au_ds = get_region_ahi_vaa(AU_lons_1km, AU_lats_1km)

In [ ]:
raa_matched_infos = []
count = 1
for modis_day in ahi_day4match_u:
    print(str(count)+'/'+str(len(ahi_day4match_u)))
    try:
        current_time = datetime.strptime(modis_day + 'T23:59:59Z', "%Y%m%dT%H:%M:%SZ")
        modis_doy = (current_time-year_first_day_date).days + 1
        modis_doy_str = (3-len(str(modis_doy)))*'0'+str(modis_doy)

        geotif_files = os.listdir(MXD_GEOTIFF_FOLDER)
        vza_regex_pattern = 'MYD03.A2018'+ modis_doy_str + r'(\S)+'+'_SensorZenith.tif'
        vza_modis_files = regex_match_substring(vza_regex_pattern, geotif_files)

        for idx in range(len(vza_modis_files)):
            vza_modis_file = vza_modis_files[idx]
            vza_modis_filename = os.path.join(MXD_GEOTIFF_FOLDER, vza_modis_file)
            file_timeflag = vza_modis_file.replace('_SensorZenith.tif', '_')
            modis_obs_time = modis_day+file_timeflag.split('.')[2]
            vaa_modis_filename = os.path.join(MXD_GEOTIFF_FOLDER, file_timeflag+'SensorAzimuth.tif')
            sza_modis_filename = os.path.join(MXD_GEOTIFF_FOLDER, file_timeflag+'SolarZenith.tif')
            saa_modis_filename = os.path.join(MXD_GEOTIFF_FOLDER, file_timeflag+'SolarAzimuth.tif')

            vza_modis_au_ds = clip_tif_studyarea_angle(vza_modis_filename, AU_lons_1km, AU_lats_1km)
            vza_modis_array = numpy.array(vza_modis_au_ds.values[0])
            if len(vza_modis_array[~numpy.isnan(vza_modis_array)]) > 0:
                print('raa-matching...')
                vaa_modis_au_ds = clip_tif_studyarea_angle(vaa_modis_filename, AU_lons_1km, AU_lats_1km)
                saa_modis_au_ds = clip_tif_studyarea_angle(saa_modis_filename, AU_lons_1km, AU_lats_1km)

                ahi_obs_timedate = nearest_10_minute_interval(modis_obs_time)
                ahi_obs_time = ahi_obs_timedate.strftime("%Y%m%d%H%M")
                ahi_sza_file = os.path.join(AHI_ANGLE_FOLDER, ahi_obs_time[:6], '4KM', ahi_obs_time[:8], ahi_obs_time+'.sun.zth.fld.4km.bin.bz2')
                ahi_saa_file = os.path.join(AHI_ANGLE_FOLDER, ahi_obs_time[:6], '4KM', ahi_obs_time[:8], ahi_obs_time+'.sun.azm.fld.4km.bin.bz2')
                saa_ahi_au_ds = get_region_ahi_saa(ahi_saa_file, AU_lons_1km, AU_lats_1km)

                modis_vza = vza_modis_array
                modis_vaa = numpy.array(vaa_modis_au_ds.values[0])
                modis_saa = numpy.array(saa_modis_au_ds.values[0])
                ahi_vza = numpy.array(vza_ahi_au_ds.values)
                ahi_vaa = numpy.array(vaa_ahi_au_ds.values)
                ahi_saa= numpy.array(saa_ahi_au_ds.values)

                # RAA-matching
                raa_match_2d = is_raa_matched(modis_vza, modis_vaa, modis_saa, ahi_vza, ahi_vaa, ahi_saa)
                raa_match_2d = raa_match_2d.astype(int)
                raa_indices = numpy.argwhere(raa_match_2d == 1)
                lon_values = AU_lons_1km[raa_indices[:, 1]]
                lat_values = AU_lats_1km[raa_indices[:, 0]]
                print(raa_match_2d.max())
                # lon lat modis_time modis_vza ahi_vza modis_vaa ahi_vaa modis_sza modis_saa
                for lon_val, lat_val in zip(lon_values, lat_values):
                    lon_v = round(lon_val,2)
                    lat_v = round(lat_val,2)
#                     if lon_v in AU_lons_10km and lat_v in AU_lats_10km:
                    lat_idx = int((-28 - lat_val)/0.01)
                    lon_idx = int((lon_val - 110)/0.01)
                    modis_vza_v = str(round(modis_vza[lat_idx][lon_idx], 2))
                    ahi_vza_v = str(round(ahi_vza[lat_idx][lon_idx], 2))
                    modis_vaa_v = str(round(modis_vaa[lat_idx][lon_idx], 2))
                    ahi_vaa_v = str(round(ahi_vaa[lat_idx][lon_idx], 2))
                    modis_saa_v = str(round(modis_saa[lat_idx][lon_idx], 2))
                    ahi_saa_v = str(round(ahi_saa[lat_idx][lon_idx], 2))
                    matched_info = [str(lon_v), str(lat_v), modis_obs_time, ahi_obs_time, modis_vza_v, ahi_vza_v, modis_vaa_v, ahi_vaa_v, modis_saa_v, ahi_saa_v]
#                     print(matched_info)
                    raa_matched_infos.append(matched_info)

                break
    except Exception as e:
        print(e)
    count+=1

In [36]:
len(raa_matched_infos)

285856

In [43]:
numpy.save('/data01/people/beichen/workspace/20240117/MYD_RAA_AU.npy', numpy.array(raa_matched_infos))

In [38]:
AU_lons_10km_str = numpy.round(AU_lons_10km, decimals=2).astype(str)
AU_lats_10km_str = numpy.round(AU_lats_10km, decimals=2).astype(str)

raa_matched_infos_10km = []
for raa_item in raa_matched_infos:
    if raa_item[0] in AU_lons_10km_str and raa_item[1] in AU_lats_10km_str:
        raa_matched_infos_10km.append(raa_item)

In [41]:
len(raa_matched_infos_10km)

2846

In [42]:
numpy.save('/data01/people/beichen/workspace/20240117/MYD_RAA_AU_10km.npy', numpy.array(raa_matched_infos_10km))